<a href="https://colab.research.google.com/github/ppaunovski/vvmn_graph_mixer/blob/master/GraphMixer_reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/CongWeilin/GraphMixer.git

fatal: destination path 'GraphMixer' already exists and is not an empty directory.


In [2]:
import torch

# pytorch
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

# pytorch-geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-{torch.__version__}.html

# pybind11 (used for c++ sampler)
!pip install pybind11

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 947.1/947.1 kB 14.7 MB/s eta 0:00:00


In [3]:
!wget -P ./REDDIT https://s3.us-west-2.amazonaws.com/dgl-data/dataset/tgl/REDDIT/edge_features.pt
!wget -P ./REDDIT https://s3.us-west-2.amazonaws.com/dgl-data/dataset/tgl/REDDIT/edges.csv
!wget -P ./REDDIT https://s3.us-west-2.amazonaws.com/dgl-data/dataset/tgl/REDDIT/labels.csv

--2024-07-10 17:05:26--  https://s3.us-west-2.amazonaws.com/dgl-data/dataset/tgl/REDDIT/edge_features.pt
Resolving s3.us-west-2.amazonaws.com (s3.us-west-2.amazonaws.com)... 52.92.227.144, 52.92.196.88, 52.92.249.88, ...
Connecting to s3.us-west-2.amazonaws.com (s3.us-west-2.amazonaws.com)|52.92.227.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 462644280 (441M) [binary/octet-stream]
Saving to: ‘./REDDIT/edge_features.pt’

edge_features.pt    100%[===================>] 441.21M  20.4MB/s    in 20s     

2024-07-10 17:05:47 (21.7 MB/s) - ‘./REDDIT/edge_features.pt’ saved [462644280/462644280]

--2024-07-10 17:05:47--  https://s3.us-west-2.amazonaws.com/dgl-data/dataset/tgl/REDDIT/edges.csv
Resolving s3.us-west-2.amazonaws.com (s3.us-west-2.amazonaws.com)... 52.92.207.208, 52.92.211.240, 52.218.178.160, ...
Connecting to s3.us-west-2.amazonaws.com (s3.us-west-2.amazonaws.com)|52.92.207.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [6]:
!python ./GraphMixer/gen_graph.py --data REDDIT

Namespace(data='REDDIT', add_reverse=True)
num_nodes:  10985
100% 672447/672447 [00:32<00:00, 20792.81it/s]
100% 10985/10985 [00:00<00:00, 388099.78it/s]
Sorting...
100% 10985/10985 [00:00<00:00, 109904.08it/s]
saving...


In [10]:
!cd GraphMixer

In [13]:
!cd GraphMixer && python setup.py build_ext --inplace


running build_ext
building 'sampler_core' extension
creating build
creating build/temp.linux-x86_64-cpython-310
x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/pybind11/include -I/usr/include/python3.10 -c sampler_core.cpp -o build/temp.linux-x86_64-cpython-310/sampler_core.o -fvisibility=hidden -g0 -fopenmp
sampler_core.cpp: In constructor ‘ParallelSampler::ParallelSampler(std::vector<int>&, std::vector<int>&, std::vector<int>&, std::vector<float>&, int, int, int, std::vector<int>&, bool, bool, int, TimeStampType)’:
sampler_core.cpp:57:10: warning: ‘ParallelSampler::prop_time’ will be initialized after []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wreorder-Wreorder]8;;]
   57 |     bool prop_time;
      |          ^~~~~~~~~
sampler_core.cpp:51:9: warning:   ‘int ParallelSampler::num_thread_per_worker’ 

In [15]:
!cd GraphMixer && python train.py --data REDDIT     --num_neighbors 10 --use_cached_subgraph --use_onehot_node_feats

Namespace(data='REDDIT', device=0, batch_size=600, epochs=300, max_edges=50, lr=0.0001, weight_decay=1e-06, model='mlp_mixer', neg_samples=1, extra_neg_samples=5, num_neighbors=10, sampled_num_hops=1, hidden_dims=100, num_layers=1, regen_models=True, check_data_leakage=False, ignore_node_feats=False, node_feats_as_edge_feats=False, ignore_edge_feats=False, use_onehot_node_feats=True, use_graph_structure=True, structure_time_gap=2000, structure_hops=1, use_node_cls=False, use_cached_subgraph=True)
Train 470713, Valid 100867, Test 100867
Num nodes 10985, num edges 672447
>>> Use one-hot node features
Node feature dim 10985, edge feature dim 172
base_model.feat_encoder.time_encoder.w.weight False
base_model.feat_encoder.time_encoder.w.bias False
base_model.feat_encoder.feat_encoder.weight True
base_model.feat_encoder.feat_encoder.bias True
base_model.layernorm.weight True
base_model.layernorm.bias True
base_model.mlp_head.weight True
base_model.mlp_head.bias True
base_model.mixer_blocks.0